In [4]:
# imports
from pokemontcgsdk import Card
from pokemontcgsdk import Set
from pokemontcgsdk import Type
from pokemontcgsdk import Supertype
from pokemontcgsdk import Subtype
from pokemontcgsdk import Rarity

In [5]:
# getting the api key
import os

api_key = os.getenv('POKEMONTCG_IO_API_KEY')

In [6]:
import pandas as pd

# Load the tab-delimited text file into a dataframe
file_path = "Example Cube.txt"
df = pd.read_csv(file_path, sep='\t')
df.columns = ["Name", "Set", "Number", "Card Type", "Sub Type", "Image URL", "Cost"]
# Display the dataframe
df.head()

,Name,Set,Number,Card Type,Sub Type,Image URL,Cost
0,Dustox,LOT,28,Pokémon,Grass,https://images.pokemontcg.io/sm8/28.png,$0.37
1,Sceptile,PRC,8,Pokémon,Grass,https://images.pokemontcg.io/xy5/8.png,$1.32
2,Victreebel,UNB,15,Pokémon,Grass,https://images.pokemontcg.io/sm10/15.png,$0.32
3,Kricketune,swsh10,10,Pokémon,Grass,https://images.pokemontcg.io/swsh10/10.png,$0.06
4,Leafeon,sv6,11,Pokémon,Grass,https://images.pokemontcg.io/sv6/11.png,$0.11


In [7]:
# testing look up
card =Card.find('swsh10-10')
print(card.name)

Kricketune


In [8]:
# appending data set availability
found_in_api = []
for index, row in df.iterrows():
    try:
        card = Card.find(row['Set'] + '-' + row['Number'])
        found_in_api.append(True)
    except:
        found_in_api.append(False)
df['Found in API'] = found_in_api
# Display the updated dataframe
df.head()

,Name,Set,Number,Card Type,Sub Type,Image URL,Cost,Found in API
0,Dustox,LOT,28,Pokémon,Grass,https://images.pokemontcg.io/sm8/28.png,$0.37,False
1,Sceptile,PRC,8,Pokémon,Grass,https://images.pokemontcg.io/xy5/8.png,$1.32,False
2,Victreebel,UNB,15,Pokémon,Grass,https://images.pokemontcg.io/sm10/15.png,$0.32,False
3,Kricketune,swsh10,10,Pokémon,Grass,https://images.pokemontcg.io/swsh10/10.png,$0.06,True
4,Leafeon,sv6,11,Pokémon,Grass,https://images.pokemontcg.io/sv6/11.png,$0.11,True


In [9]:
# percent found in api
total_cards = len(df)
found_cards = df['Found in API'].sum()
percent_found = (found_cards / total_cards) * 100
print("Percent Found:", percent_found, "%")

Percent Found: 46.846846846846844 %


In [10]:
# extracting id base on URL
def extract_id(url):
    split_url = url.split(".")[-2].split("/")
    return split_url[-2] + "-" + split_url[-1]

df["ID"] = df["Image URL"].apply(extract_id)
df.head()

,Name,Set,Number,Card Type,Sub Type,Image URL,Cost,Found in API,ID
0,Dustox,LOT,28,Pokémon,Grass,https://images.pokemontcg.io/sm8/28.png,$0.37,False,sm8-28
1,Sceptile,PRC,8,Pokémon,Grass,https://images.pokemontcg.io/xy5/8.png,$1.32,False,xy5-8
2,Victreebel,UNB,15,Pokémon,Grass,https://images.pokemontcg.io/sm10/15.png,$0.32,False,sm10-15
3,Kricketune,swsh10,10,Pokémon,Grass,https://images.pokemontcg.io/swsh10/10.png,$0.06,True,swsh10-10
4,Leafeon,sv6,11,Pokémon,Grass,https://images.pokemontcg.io/sv6/11.png,$0.11,True,sv6-11


In [11]:
found_in_api = []
cards = []
for index, row in df.iterrows():
    try:
        card = Card.find(row['ID'])
        found_in_api.append(True)
        cards.append(card)
    except:
        try:
            card = Card.find(row['Set'] + '-' + row['Number'])
            found_in_api.append(True)
            cards.append(card)
        except:
            found_in_api.append(False)
            cards.append(None)
df['Found in API'] = found_in_api

# Display the updated search based on id
total_cards = len(df)
found_cards = df['Found in API'].sum()
percent_found = (found_cards / total_cards) * 100
print("Percent Found:", percent_found, "%")

Percent Found: 100.0 %


In [12]:
df.head()

,Name,Set,Number,Card Type,Sub Type,Image URL,Cost,Found in API,ID
0,Dustox,LOT,28,Pokémon,Grass,https://images.pokemontcg.io/sm8/28.png,$0.37,True,sm8-28
1,Sceptile,PRC,8,Pokémon,Grass,https://images.pokemontcg.io/xy5/8.png,$1.32,True,xy5-8
2,Victreebel,UNB,15,Pokémon,Grass,https://images.pokemontcg.io/sm10/15.png,$0.32,True,sm10-15
3,Kricketune,swsh10,10,Pokémon,Grass,https://images.pokemontcg.io/swsh10/10.png,$0.06,True,swsh10-10
4,Leafeon,sv6,11,Pokémon,Grass,https://images.pokemontcg.io/sv6/11.png,$0.11,True,sv6-11


In [13]:
import pickle

# saving the dataframe and the card information
with open('example_cube_cards.pkl', 'wb') as f:
    pickle.dump(cards, f)

df.to_csv('example_cube.csv', index=False)

In [14]:
# reloading the data
import pickle
import pandas as pd

df = pd.read_csv('example_cube.csv')
with open("example_cube_cards.pkl", 'rb') as f:
    cards = pickle.load(f)


In [15]:
# testing 
attack_dict = {"Cost": [], "damage": [], "text": [], "stage": []}
for card in cards:
    if card is not None and card.attacks is not None:
        for attack in card.attacks:
            attack_dict["Cost"].append(attack.convertedEnergyCost)
            attack_dict["damage"].append(attack.damage)
            attack_dict["text"].append(attack.text)
            stage = "Basic"
            if "Stage 2" in card.supertype:
                stage = "Stage 2"
            elif "Stage 1" in card.supertype:
                stage = "Stage 1"
            attack_dict["stage"].append(stage)
attack_df = pd.DataFrame(attack_dict)
attack_df.head()

,Cost,damage,text,stage
0,2,70,,Basic
1,3,70,,Basic
2,1,10+,This attack does 60 more damage for each Speci...,Basic
3,3,90,The Defending Pokémon has no Abilities until t...,Basic
4,2,50,,Basic


In [16]:
import json

# Convert card object to JSON for testing

# Custom serialization function for Card objects
def card_serializer(card):
	return card.__dict__

card_json = json.dumps(cards[0], default=card_serializer, indent=4)
print(card_json)

{
    "abilities": [
        {
            "name": "Hazardous Evolution",
            "text": "When you play this Pok\u00e9mon from your hand to evolve 1 of your Pok\u00e9mon during your turn, you may leave your opponent's Active Pok\u00e9mon Paralyzed and Poisoned. If you do, put 3 damage counters instead of 1 on that Pok\u00e9mon between turns.",
            "type": "Ability"
        }
    ],
    "artist": "Midori Harada",
    "ancientTrait": null,
    "attacks": [
        {
            "name": "Gust",
            "cost": [
                "Grass",
                "Colorless"
            ],
            "convertedEnergyCost": 2,
            "damage": "70",
            "text": ""
        }
    ],
    "cardmarket": {
        "url": "https://prices.pokemontcg.io/cardmarket/sm8-28",
        "updatedAt": "2025/04/08",
        "prices": {
            "averageSellPrice": 0.39,
            "lowPrice": 0.05,
            "trendPrice": 0.61,
            "germanProLow": 0.0,
            "suggeste